In [74]:
# !pip install pandas
# !pip install unicodedata

In [1]:
import requests as rq
from bs4 import BeautifulSoup as bs
import pandas as pd
import os
from unicodedata import normalize
import re
from bs4.element import Comment

In [77]:
# url = 'https://www.google.com.br/search?q=eleições+2018&ei=eMuaW4fhNsn7wQTR5L_gAg&start=%d&sa=N&biw=1280&bih=496'
# url = 'https://www.google.com.br/search?q=ciencia+de+dados&ei=rxqhW_vNK4PAwASJ-Y2IAw&start=%d&sa=N&biw=1280&bih=496'
# url = 'https://www.google.com.br/search?q=vendas+online&ei=eMuaW4fhNsn7wQTR5L_gAg&start=%d&sa=N&biw=1280&bih=496'

# search = []
# for i in range(1):
#     page = rq.get(url % (i*10))
#     print (page)
#     page = bs(page.text, 'html.parser')
# #     print (page)
# #     print (page.prettify("UTF-8"))
#     search += page.find_all('h3', 'r')
#     nextPage = page.find('div', id = 'foot').find('a')
   
# getLinks = lambda p: dict({ 'title': p.find('a').text, 'link': p.find('a').get('href') })

# links = [getLinks(p) for p in search]

# print (search)

page = rq.get("https://www1.folha.uol.com.br/mercado/2018/07/vendas-online-levam-shoppings-a-se-reinventar-e-focar-em-logistica.shtml")

print (page.status_code)
print (page.ok)


# file = open('Documents/vendaonline.csv', 'w')
# for link in links:
#     file.write('%s; %s\n' % (link['title'], link['link']))
# file.close()

200
True


In [68]:
# data_set = pd.read_csv('./Documents/vendaonline.csv', sep=';', header = None)
# data_set = pd.read_csv('./Documents/ciencia de dados.csv', sep=';', header = None)
data_set = pd.read_csv('./Documents/eleicao2018.csv', sep=';', header = None)

data_set.columns = ["Name","URL"]
data_set.URL = data_set["URL"].str.split('=').str[1]
data_set.URL = data_set["URL"].str.split('&').str[0]
# data_set.to_csv("./Documents/vendasonlinenamelink.csv",sep=";", index=False)
data_set.to_csv("./Documents/eleicao 2018 name link.csv",sep=";", index=False)

In [72]:

# data_set = pd.read_csv("./Documents/vendasonlinenamelink.csv", sep=";")
# data_set = pd.read_csv("./Documents/ciencia de dados name link.csv", sep=";")
data_set = pd.read_csv("./Documents/eleicao 2018 name link.csv", sep=";")

for index, link in enumerate(data_set.URL.values):
    print (index, link)
    page = rq.get(link)
    if (page.ok):
#         f = open("./Pages/vendas online/" + str(index) + ".html", 'w')
#         f = open("./Pages/ciencia de dados/" + str(index) + ".html", 'w')
        f = open("./Pages/eleicoes 2018/" + str(index + 110) + ".html", 'w')
        f.write(page.text)
        f.close
#     print (index, link)

0 https://www.jmnoticia.com.br/2018/09/18/tre-to-finaliza-julgamento-dos-registros-de-candidaturas-para-as-eleicoes-2018/
1 https://www.destakjornal.com.br/brasil/eleicoes-2018/detalhe/veja-onde-estao-os-presidenciaveis-nesta-terca-feira-18
2 https://voltemosaoevangelho.com/blog/2018/09/eleicoes-2018-carta-aberta-a-igreja-brasileira/
3 https://entretenimento.oportaln10.com.br/roda-viva-17-09-2018-na-serie-desafios-eleicoes-2018-um-debate-sobre-economia-82403/
4 http://investimentosenoticias.com.br/eleicoes/eleicoes-2018-brasil
5 http://www.vidadeturista.com/artigos/eleicoes-2018-justificativa-eleitoral-em-viagem.html
6 https://www.boatos.org/politica/maria-do-rosario-bolsonaro-2018.html
7 http://www.tre-se.jus.br/imprensa/noticias-tre-se/2018/Junho/eleicoes-2018-fique-de-olho-nos-prazos
8 http://www.portaldaindustria.com.br/cni/canais/propostas-da-industria-para-eleicoes-2018/
9 http://www.cnbb.org.br/eleicoes-2018-compromisso-e-esperanca/
10 http://www.tnh1.com.br/video/vid/1o-reporta

In [16]:
def remover_acentos_e_pontuacao(txt):
    sem_acentos = normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')
    return re.sub('[^a-zA-Z0-9 ]', '', sem_acentos)


def tag_visible(element):
    
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

# utilizado para pegar o titulo da página
def title_page(element):
    
    if element.parent.name in ['title']:
        return True
    return False

# utilizado para pegar apenas o texto da página html
def getText(arquivo):        
    bsFile = bs(open(arquivo, encoding="UTF-8"),"html.parser")
    texts = bsFile.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    title = filter(title_page, texts)
    words = [t.strip() for t in visible_texts] # pega as palavras da página
    words_title = [t.strip() for t in title] # pega o titulo da página
    words = filter(lambda x: len(x)>0, words) # filtra apenas as palavras que tem pelo menos 1 caracter
    words_title = filter(lambda x: len(x)>0, words_title)
                       
#     words += words_title 
    text = " ".join(words)
    title = " ".join(words_title)
    return {
        "text": text,
        "title": title
    }

In [19]:
folderRead = "./Pages/eleicoes 2018/"
fodlerWrite = "./Text 2/Eleicoes 2018/"

# folderRead = "./Pages/ciencia de dados/"
# fodlerWrite = "./Text 2/Ciencia de dados/"

# folderRead = "./Pages/vendas online/"
# folderWrite = "./Text 2/Vendas online/"

files = os.listdir(folderRead)

for index, file in enumerate(files):
    print (file)
    if file.endswith(".html"):
        page = getText(folderRead + file)
        saveFile = open(folderWrite + str(index) + ".txt", "w", encoding="UTF-8")
        saveFile.write(page["title"] + "\n" + page["text"])
        saveFile.close()

0.html
1.html
10.html
100.html
101.html
102.html
103.html
104.html
105.html
106.html
11.html
110.html
111.html
112.html
113.html
114.html
115.html
116.html
117.html
118.html
12.html
120.html
13.html
14.html
15.html
16.html
17.html
18.html
19.html
2.html
20.html
21.html
23.html
24.html
26.html
27.html
28.html
29.html
3.html
30.html
31.html
32.html
33.html
34.html
35.html
36.html
37.html
38.html
39.html
4.html
40.html
41.html
42.html
44.html
45.html
46.html
47.html
49.html
5.html
50.html
51.html
52.html
53.html
54.html
55.html
6.html
60.html
61.html
62.html
63.html
64.html
66.html
67.html
68.html
69.html
7.html
70.html
71.html
72.html
73.html
74.html
75.html
76.html
77.html
78.html
79.html
8.html
80.html
81.html
82.html
83.html
84.html
85.html
86.html
87.html
89.html
9.html
90.html
91.html
92.html
93.html
94.html
96.html
97.html
98.html
99.html
